In [1]:
%pip install SDV

Note: you may need to restart the kernel to use updated packages.


In [36]:
from sdv.metadata import MultiTableMetadata
from sdv.multi_table import HMASynthesizer
from sdv.datasets.local import load_csvs
import pandas as pd

df = pd.read_csv('data3/Identity.csv')

#df['Value'] = 123
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df.to_csv('data3/Identity.csv', index = False)

metadata = MultiTableMetadata()
metadata.detect_from_csvs(
    folder_name='data3'
)

metadata.update_column(
    table_name='Contractor',
    column_name ='ContractorId',
    sdtype='id'
)
metadata.update_column(
    table_name='Contractor',
    column_name ='SourceId',
    sdtype='id'
)

metadata.update_column(
    table_name='Competence',
    column_name ='CompetenceId',
    sdtype='id'
)
metadata.update_column(
    table_name='Competence',
    column_name ='SourceId',
    sdtype='id'
)
metadata.update_column(
    table_name='Competence',
    column_name ='Contractor_Id',
    sdtype='id'
)

#metadata.update_column(
 #   table_name='Competence',
 #   column_name ='Expires',
 #   sdtype='datetime',
 #   datetime_format='%Y-%m-%d %H:%M:%S.%f'
#)


metadata.update_column(
    table_name='Identity',
    column_name='DeletedDate',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Identity',
    column_name='Value',
    sdtype='unknown',
    pii= True
)
metadata.update_column(
    table_name='Identity',
    column_name='ActivatedDate',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Contractor',
    column_name='ContractorUpdated',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Competence',
    column_name='Acquired',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Competence',
    column_name='Created',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Identity',
    column_name='Value',
    sdtype='numerical'
)

# Validate and visualize metadata
print("Validate state: ", metadata.validate())
#metadata.visualize(
#    show_table_details='full',
#    show_relationship_labels=True,
#    output_filepath='my_metadata3.png'
#)
metadata.save_to_json('TestdataJson5')

real_data = load_csvs(
    folder_name='data3',
    read_csv_parameters={
        'skipinitialspace': True,
        'encoding': 'utf_8'
    }
)

print("Original Contractor:\n", real_data['Contractor'].head())
print("Original Competence:\n", real_data['Competence'].head())
print("Original Identity:\n", real_data['Identity'].head())

#real_data['Identity']['Value'] = pd.to_numeric(real_data['Identity']['Value'], errors='coerce')

# Drop rows with NaN values in 'Value' column
#real_data['Identity'] = real_data['Identity'].dropna(subset=['Value'])

# Identify valid contractor IDs
#valid_contractor_ids = set(real_data['Contractor']['Contractor_Id'])

# mismatched IDs i Competence Identity
#mismatched_competence_ids = set(real_data['Competence']['Contractor_Id']) - valid_contractor_ids
#mismatched_identity_ids = set(real_data['Identity']['Contractor_Id']) - valid_contractor_ids

# Clean the data by removing rows with invalid foreign keys
#real_data['Competence'] = real_data['Competence'][real_data['Competence']['Contractor_Id'].isin(valid_contractor_ids)]
#real_data['Identity'] = real_data['Identity'][real_data['Identity']['Contractor_Id'].isin(valid_contractor_ids)]

# Check if any of the tables are empty after cleaning
if real_data['Contractor'].empty:
    print("Contractor is empty")
if real_data['Competence'].empty:
    print("Competence is empty")
if real_data['Identity'].empty:
    print("Identity is empty")

synthesizer = HMASynthesizer(metadata)
synthesizer.fit(real_data)

synthetic_data = synthesizer.sample(scale=1)
print("Synthetic Contractor:\n", synthetic_data['Contractor'].head())
print("Synthetic Competence:\n", synthetic_data['Competence'].head())
print("Synthetic Identity:\n", synthetic_data['Identity'].head())


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sdv/datasets/local.py:34: UserWarning: Ignoring incompatible files ['.amlignore', '.amlignore.amltmp', 'competence.csv.amltmp', 'identity.csv.amltmp'] in folder 'data3'.
  warnings.warn(
Preprocess Tables: 100%|██████████| 3/3 [00:00<00:00,  5.47it/s]
(1/2) Tables 'Contractor' and 'Competence' ('Contractor_Id'): 100%|██████████| 14/14 [00:06<00:00,  2.04it/s]
(2/2) Tables 'Contractor' and 'Identity' ('Contractor_Id'): 100%|██████████| 8/8 [00:01<00:00,  4.35it/s]
Modeling Tables: 100%|██████████| 1/1 [00:05<00:00,  5.99s/it]


Validate state:  None
Original Contractor:
    Contractor_Id                          ContractorId  Source  SourceId  \
0         405298  d2cb31ab-c48f-4601-baf7-0001afe8e75c       0    151748   
1         405299  3fdf03f8-230e-46ac-9b12-0001c95595e3       0     40493   
2         405300  53dab2b6-5d3b-4b95-9638-0002199a45cb       0     78347   
3         405302  9af74bcc-f888-4ae9-a3ea-0004195bf914       0    304775   
4         405304  b7e66908-571f-4f36-b9ae-00058459c79c       0     32093   

   FirstName  LastName                                           PhotoUrl  \
0     Leonel     Nohel  https://ssgcardissuestaging.blob.core.windows....   
1     Hassan     Leafe  https://ssgcardissuestaging.blob.core.windows....   
2  Jefferson  Crivelli  https://ssgcardissuestaging.blob.core.windows....   
3       Rolf      Muns  https://ssgcardissuestaging.blob.core.windows....   
4       Vito    Morles  https://ssgcardissuestaging.blob.core.windows....   

   Nationality  BirthDay            

In [37]:
import uuid

def format_columns_as_uuids(table, column_names):
    for column in column_names:
        table[column] = [str(uuid.uuid4()) for _ in range(len(table))]
    return table

synthetic_data['Contractor'] = format_columns_as_uuids(synthetic_data['Contractor'], ['ContractorId'])
synthetic_data['Competence'] = format_columns_as_uuids(synthetic_data['Competence'], ['CompetenceId', 'SourceId'])
synthetic_data['Competence']['CreatedBy'] = 'HMASynthesizer'
synthetic_data['Contractor']['PhotoUrl'] = 'HMASynthesizer'


In [42]:
def modify_string(value):
    value_str = str(value)
    return value_str[:-2] + 'hma'

synthetic_data['Identity']['Value'] = synthetic_data['Identity']['Value'].apply(modify_string)


In [44]:
synthetic_data['Competence'].to_csv('competence_HMAdata.csv', index=False)
synthetic_data['Contractor'].to_csv('contractor_HMAdata.csv', index=False)
synthetic_data['Identity'].to_csv('identity_HMAdata.csv', index=False)